In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.datasets.mnist import load_data

import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

In [3]:
def get_weight_variable(shape):
    weights = tf.get_variable("weights", shape, initializer = tf.truncated_normal_initializer(stddev=0.1))
    return weights

In [4]:
def inference(input_tensor):
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE])
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer = tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE])
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer = tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    
    return layer2

In [5]:
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.005
LEARNING_RATE_DECAY = 0.99
TRAINING_STEPS = 6000

In [6]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None,INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None,OUTPUT_NODE], name="y-input")
    
    y = inference(x)
    train_size = mnist[0][0].shape[0]
    mnist_x = np.reshape(mnist[0][0],[-1,784])
    mnist_y = np.eye(10)[mnist[0][1]]
    
    test_feed = {x: np.reshape(mnist[1][0],[-1,784]),y_: np.eye(10)[mnist[1][1]]}
    
    global_step = tf.Variable(0, trainable=False)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    loss = tf.reduce_mean(cross_entropy)
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step,train_size / BATCH_SIZE, LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
    
        for i in range(TRAINING_STEPS):
            xs, ys = mnist_x[i*BATCH_SIZE%train_size:(i+1)*BATCH_SIZE%train_size], mnist_y[i*BATCH_SIZE%train_size:(i+1)*BATCH_SIZE%train_size]
            _, loss_value, step = sess.run([train_step, loss, global_step], feed_dict={x: xs, y_: ys})
            
            if i%100 == 0:
                test_acc = sess.run(accuracy, feed_dict=test_feed)
                print("After {} training steps, loss on training batch is {},test accuracy is {}".format(step, loss_value, test_acc))

In [7]:
mnist = load_data("mnist.npz")
train(mnist)

writer = tf.summary.FileWriter("./log",tf.get_default_graph())
writer.close()

After 1 training steps, loss on training batch is 410.2440490722656,test accuracy is 0.19830000400543213
After 101 training steps, loss on training batch is 1.235223412513733,test accuracy is 0.6680999994277954
After 201 training steps, loss on training batch is 1.5397768020629883,test accuracy is 0.7210000157356262
After 301 training steps, loss on training batch is 1.7024928331375122,test accuracy is 0.7469000220298767
After 401 training steps, loss on training batch is 1.144922137260437,test accuracy is 0.7886000275611877
After 501 training steps, loss on training batch is 0.924815833568573,test accuracy is 0.8083999752998352
After 601 training steps, loss on training batch is 0.49199581146240234,test accuracy is 0.8385000228881836
After 701 training steps, loss on training batch is 0.5883651375770569,test accuracy is 0.8492000102996826
After 801 training steps, loss on training batch is 0.5868187546730042,test accuracy is 0.8640000224113464
After 901 training steps, loss on trainin